<a href="https://colab.research.google.com/github/yklove1025/Ensamble-Sentiment-Analysis-for-Korean-English/blob/main/Sentiment_analysis_korean_KSB_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **앙상블 한국어 감정분석 모델**

본 한국어 감정분석 모델은 고려대학교 디지털금융공학과 DFE610 디지털금융공학을 위한 자연어처리기술 수업의 과제를 수행하기 위해 개발한 모델이다.

본 앙상블 한국어 감정분석 모델은 앙상블 감정분석 대상 모델로 본 과정 수업 실습자료인 'Transformer를 이용한 감정분석 모델', 'CNN을 이용한 감정분석 모델' 과 '네이버 영화리뷰 감정분석 with Hugging Face BERT' 모델을 실습자료로 제공된 소스와 오픈소스를 참고하여 구성하였다.


1.   모델의 오픈소스 참고 출처
- 모델의 오픈소스 참고 출처



> Transformer를 이용한 감정분석 모델 : Transformer를 이용한 감정분석 모델 : https://github.com/Parkchanjun/KU-NLP-2020-1, [5] Transformer를_이용한_감정분석_한국어.ipynb


> CNN을 이용한 감정분석 모델 : https://github.com/Parkchanjun/KU-NLP-2020-1, [3] CNN_Sentiment_Analysis.ipynb


> 네이버 영화리뷰 감정분석 with Hugging Face BERT : https://colab.research.google.com/drive/13AMh8N9tEIw5rmxgc1fQfS8581mWegxj


2. 상세사항
- 아래 깃허브의 Readme.md 확인

  https://github.com/yklove1025/Ensamble-Sentiment-Analysis-for-Korean

3. 실행환경
- 본 소스는 구글 Colab에서 작성됨 
- 본 소스의 실행을 위해서는 Colab Pro 환경이 필요(GPU 메모리 필요)
- ① 실행 후 런타임 초기화 ② 순차 실행 → 종료 후 ③ 실행 (실행순번 표기)
- BERT 모델 저장 후 업로드 시에도 메모리 문제 발생 가능함(재실행 권장) 


**Hugging Face의 Transformer Model 설치 ①,②**

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 11.4MB/s 
     |████████████████████████████████| 890kB 48.6MB/s 
     |████████████████████████████████| 2.9MB 38.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3d60ed2198d91ac5d0f2cb4f6c17a4c89124aee95ce6ce370496fabaecb0b761
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


**한국어 형태소 분석기(Konlpy) 설치 ①,②**

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 64.3MB/s 
     |████████████████████████████████| 460kB 43.7MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


**학습 데이터 다운로드 (Naver, NSMC) ①,②**

In [ ]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 7.44 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


**Glove 다운로드 ①**

In [ ]:
#Glove 다운로드
#출처: https://nlp.stanford.edu/projects/glove/
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-12-20 00:53:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-20 00:53:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-20 00:53:36--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

**공통 패키지 Import ①,②**

BERT, CNN, Transformer 모듈에 필요한 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
import torch
import tensorflow as tf
import keras
import os
import nltk
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from keras.models import Sequential
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Komoran
from konlpy.tag import Twitter

**데이터 로드 ①,②** 

In [ ]:
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


**테스트, 트레이닝 변수 선언 ①,②**

In [ ]:
X_train = []
X_test = []
okt = Okt()

**트레이닝, 테스트 Document 형태소 분리 전처리 ①**

In [ ]:
sentences = train['document']

X_train = [okt.morphs(word) for word in sentences.astype(str)]
print('x_train : ', X_train[:10])

x_train :  [['아', '더빙', '..', '진짜', '짜증나네요', '목소리'], ['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나'], ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'], ['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정'], ['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다'], ['막', '걸음', '마', '뗀', '3', '세', '부터', '초등학교', '1', '학년', '생인', '8', '살용', '영화', '.', 'ㅋㅋㅋ', '...', '별', '반개', '도', '아까', '움', '.'], ['원작', '의', '긴장감', '을', '제대로', '살려내지못', '했다', '.'], ['별', '반개', '도', '아깝다', '욕', '나온다', '이응경', '길용우', '연', '기', '생활', '이', '몇', '년', '인지', '..', '정말', '발', '로', '해도', '그것', '보단', '낫겟다', '납치', '.', '감금', '만', '반복', '반복', '..', '이', '드라마', '는', '가족', '도', '없다', '연기', '못', '하는', '사람', '만', '모', '엿', '네'], ['액션', '이', '없는데도', '재미', '있는', '몇', '안되는', '영화'], ['왜케', '평점', '이', '낮은건데', '?', '꽤', '볼', '만', '한', '데', '..', '헐리우드', '식', '화려함에만', '너무', '길들여져', '있나', '?']]


In [ ]:
sentences = test['document']

X_test = [okt.morphs(word) for word in sentences.astype(str)]
print('x_test : ', X_test[:10])

x_test :  [['굳', 'ㅋ'], ['GDNTOPCLASSINTHECLUB'], ['뭐', '야', '이', '평점', '들', '은', '....', '나쁘진', '않지만', '10', '점', '짜', '리', '는', '더', '더욱', '아니잖아'], ['지루하지는', '않은데', '완전', '막장', '임', '...', '돈', '주고', '보기', '에는', '....'], ['3', 'D', '만', '아니었어도', '별', '다섯', '개', '줬을텐데', '..', '왜', '3', 'D', '로', '나와서', '제', '심기', '를', '불편하게', '하죠', '??'], ['음악', '이', '주가', '된', ',', '최고', '의', '음악', '영화'], ['진정한', '쓰레기'], ['마치', '미국', '애니', '에서', '튀어나온듯', '한', '창의력', '없는', '로봇', '디자인', '부터가', ',', '고개', '를', '젖게', '한다'], ['갈수록', '개판', '되가는', '중국영화', '유치하고', '내용', '없음', '폼', '잡다', '끝남', '말', '도', '안되는', '무기', '에', '유치한', 'cg', '남무', '아', '그립다', '동사서독', '같은', '영화', '가', '이건', '3', '류', '아', '류작', '이다'], ['이별', '의', '아픔', '뒤', '에', '찾아오는', '새로운', '인연', '의', '기쁨', 'But', ',', '모든', '사람', '이', '그렇지는', '않네', '..']]


In [ ]:
print(np.shape(X_test))
print(np.shape(X_train))

(50000,)
(150000,)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


- Lable data set ①,②

In [ ]:
labels_train = train['label'].values
labels_test = test['label'].values

**Transformer, CNN 모델 텍스트 데이터 패딩 및 문장 처리 ①**

In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train) #단어 인덱스를 구축합니다.
tokenizer.fit_on_texts(X_test) 

X_train = tokenizer.texts_to_sequences(X_train) #문자열을 정수 인덱스의 리스트로 변환합니다.
X_test = tokenizer.texts_to_sequences(X_test) 

maxlen = 128  
vocab_size = len(tokenizer.word_index)+1 # 단어의 수

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)#패딩
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

EMBEDDING_DIM = 50 #100
embedding_matrix = create_embedding_matrix(
    './glove.6B.50d.txt',
    tokenizer.word_index, EMBEDDING_DIM
    )

**BERT 모델 텍스트 전처리 ①,②**

In [ ]:
sentences = train['document']

X_train_bert = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
X_train_bert[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]']

In [ ]:
sentences = test['document']

X_test_bert = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
X_test_bert[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] GDNTOPCLASSINTHECLUB [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? [SEP]',
 '[CLS] 음악이 주가 된, 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. [SEP]']

**앙상블 테스트를 위한 모델 정확도 산출 함수 선언(1만개 테스트 데이터 대상) ①,②**

In [ ]:
def convert_acc (x) :

    acc = []

    for i, row in x.iterrows() :
      if x.at[i, 'Predicted'] == x.at[i, 'label'] : i = 1 
      else : i = 0
      acc.append(i)   

    return acc

**Transformer를 이용한 감정분석 모델 설정 ①**

In [ ]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) #Multihead Attn 블록
        attn_output = self.dropout1(attn_output, training=training) #드롭아웃
        out1 = self.layernorm1(inputs + attn_output) #LM + Residual
        ffn_output = self.ffn(out1) #FF 블록
        ffn_output = self.dropout2(ffn_output, training=training) #드롭아웃
        return self.layernorm2(out1 + ffn_output) #LM + Residual

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1) #포지션 정보
        positions = self.pos_emb(positions) #포지션 임베딩
        x = self.token_emb(x) #토큰임베딩
        return x + positions #합치기

In [ ]:
embed_dim = 32  # Embedding size for each token
# embed_dim = 512  # Embedding size for each token
num_heads = 2  # Number of attention heads
# num_heads = 8  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
# ff_dim = 2048  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,)) #처음 입력
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim) #객체 생성
x = embedding_layer(inputs)  #포지셔널 임베딩
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim) #객체 생성
x = transformer_block(x) #트랜스포머 
x = layers.GlobalAveragePooling1D()(x) #Average Pooling
x = layers.Dropout(0.1)(x) #드롯아웃
x = layers.Dense(20, activation="relu")(x) #FFNN
x = layers.Dropout(0.1)(x) #드롭아웃
outputs = layers.Dense(2, activation="softmax")(x) #Softmax

model = keras.Model(inputs=inputs, outputs=outputs) #모델 생성

**Transformer를 이용한 감정분석 모델 학습 실행 ①**

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, labels_train, batch_size=32, epochs=5, validation_split=0.2)

Epoch 1/5
3750/3750 [==============================] - 93s 24ms/step - loss: 0.5102 - accuracy: 0.7053 - val_loss: 0.3398 - val_accuracy: 0.8496
Epoch 2/5
3750/3750 [==============================] - 90s 24ms/step - loss: 0.2622 - accuracy: 0.8941 - val_loss: 0.3442 - val_accuracy: 0.8581
Epoch 3/5
3750/3750 [==============================] - 90s 24ms/step - loss: 0.1841 - accuracy: 0.9270 - val_loss: 0.3830 - val_accuracy: 0.8510
Epoch 4/5
3750/3750 [==============================] - 90s 24ms/step - loss: 0.1404 - accuracy: 0.9467 - val_loss: 0.4519 - val_accuracy: 0.8457
Epoch 5/5
3750/3750 [==============================] - 90s 24ms/step - loss: 0.1152 - accuracy: 0.9558 - val_loss: 0.4783 - val_accuracy: 0.8394


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
token_and_position_embedding (None, 128, 32)           3915744   
_________________________________________________________________
transformer_block (Transform (None, 128, 32)           6464      
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0     

**Transformer을 이용한 감정분석 모델 저장 및 테스트 ①**

In [ ]:
#model.save('Transformer_SENTIMENT_ANALYSIS_KOR') #모델 저장하기

#모델 평가
test_loss,test_acc=model.evaluate(X_test,labels_test)
print("Test_acc: ",test_acc)

1563/1563 [==============================] - 6s 4ms/step - loss: 0.4899 - accuracy: 0.8382
Test_acc:  0.8381800055503845


**Transformer를 이용한 감정분석 모델 테스트 Output 데이터 생성 ①**


- 테스트 데이터 분리 (코랩 GPU 메모리 문제로 분할 실행)


In [ ]:
# test 데이터 분리 (Colab GPU 활용 제한으로 분리하여 예측 결과 저장 후 병합)
X_test_input = X_test[:10000]

- 레이블 데이터 분리 (앙상블 아웃풋 Accuracy 산출 용 레이블 데이터 분리 생성)

In [ ]:
labels_test_input = labels_test[:10000]

- 테스트 데이터로 예측 아웃풋 산출 수행

In [ ]:
x_predict_transformer = ()
x_predict_transformer = model.predict(X_test_input)

x_predict_transformer = np.argmax(x_predict_transformer, axis = 1)
x_predict_transformer = x_predict_transformer.tolist()

x_document = []
x_document = test['document']
x_document = x_document[:10000]
x_document = x_document.tolist()

x_output_transformer = pd.DataFrame({'document' : x_document, 'predict' : x_predict_transformer})
print('아웃풋 데이터 : ', x_output_transformer[:20])
print('아웃풋 데이터 Shape: ', np.shape(x_output_transformer))

아웃풋 데이터 :                                               document  predict
0                                                 굳 ㅋ        1
1                                GDNTOPCLASSINTHECLUB        0
2              뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아        0
3                    지루하지는 않은데 완전 막장임... 돈주고 보기에는....        0
4   3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??        0
5                                  음악이 주가 된, 최고의 음악영화        1
6                                             진정한 쓰레기        0
7            마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다        0
8   갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...        0
9      이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..        1
10                                   괜찮네요오랜만포켓몬스터잼밌어요        1
11                        한국독립영화의 한계 그렇게 아버지가 된다와 비교됨        0
12  청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 ...        1
13                   눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다.        1
14  "스토리, 연출, 연기, 비주얼 등 영화의 기본 조차 안된 영화에 무슨 

- Transformer 모델 예측 결과 스코어링

In [ ]:
print("Transformer model accuracy : ", accuracy_score(labels_test_input, x_predict_transformer))
print("Transformer model recall : ",recall_score(labels_test_input, x_predict_transformer))
print("Transformer model precision : ",precision_score(labels_test_input, x_predict_transformer))
print("Transformer model f1 : ",f1_score(labels_test_input, x_predict_transformer))

Transformer model accuracy :  0.8405
Transformer model recall :  0.8263189093064611
Transformer model precision :  0.8538178848509596
Transformer model f1 :  0.839843357766844


- Transformer 예측결과 CSV 파일 저장 ① 
- Colab GPU 사용량 문제로 Transformer, CNN, BERT 예측결과 로컬 PC 저장 후 로딩하여 앙상블 결과 산출

In [ ]:
ensemble_transformer = pd.DataFrame({'transformer prediction' : x_predict_transformer})
print(ensemble_transformer)

      transformer prediction
0                          1
1                          0
2                          0
3                          0
4                          0
...                      ...
9995                       1
9996                       0
9997                       0
9998                       0
9999                       0

[10000 rows x 1 columns]


In [ ]:
ensemble_transformer.to_csv('ensemble_transformer.csv')

**CNN을 이용한 감정분석 모델 설정 ①**


In [ ]:
seq_input = layers.Input(shape=(maxlen,), dtype='int32')

seq_embedded = layers.Embedding(vocab_size, 
                           EMBEDDING_DIM, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=True)(seq_input)

filters = [2,3,4,5]
conv_models = []
for filter in filters:
  conv_feat = layers.Conv1D(filters=100, 
                            kernel_size=filter, 
                            activation='relu',
                            padding='valid')(seq_embedded) #Convolution Layer
  pooled_feat = layers.GlobalMaxPooling1D()(conv_feat) #MaxPooling
  conv_models.append(pooled_feat)

conv_merged = layers.concatenate(conv_models, axis=1) #filter size가 2,3,4,5인 결과들 Concatenation

model_output = layers.Dropout(0.2)(conv_merged)
model_output = layers.Dense(10, activation='relu')(model_output)
logits = layers.Dense(1, activation='sigmoid')(model_output)

model = Model(seq_input, logits) #(입력,출력)

**CNN을 이용한 감정분석 모델 학습 실행 ①**

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

#학습 시작
history = model.fit(X_train, labels_train, epochs=5, verbose=True, validation_data=(X_test, labels_test),batch_size=32)

Epoch 1/5
4688/4688 [==============================] - 204s 43ms/step - loss: 0.4526 - accuracy: 0.7808 - val_loss: 0.3307 - val_accuracy: 0.8561
Epoch 2/5
4688/4688 [==============================] - 195s 42ms/step - loss: 0.2378 - accuracy: 0.9058 - val_loss: 0.3273 - val_accuracy: 0.8606
Epoch 3/5
4688/4688 [==============================] - 195s 42ms/step - loss: 0.1398 - accuracy: 0.9488 - val_loss: 0.4016 - val_accuracy: 0.8540
Epoch 4/5
4688/4688 [==============================] - 198s 42ms/step - loss: 0.0783 - accuracy: 0.9720 - val_loss: 0.4882 - val_accuracy: 0.8480
Epoch 5/5
4688/4688 [==============================] - 197s 42ms/step - loss: 0.0467 - accuracy: 0.9830 - val_loss: 0.6001 - val_accuracy: 0.8466


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 128, 50)      6111950     input_2[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 127, 100)     10100       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 126, 100)     15100       embedding_2[0][0]                
____________________________________________________________________________________________

**CNN을 이용한 감정분석 모델 저장 및 테스트 ①**

In [ ]:
# model.save('CNN_SENTIMENT_ANALYSIS_KOR') #모델 저장하기

#모델 평가
test_loss,test_acc=model.evaluate(X_test,labels_test)
print("Test_acc: ",test_acc)

1563/1563 [==============================] - 4s 3ms/step - loss: 0.6001 - accuracy: 0.8466
Test_acc:  0.846560001373291


**CNN를 이용한 감정분석 모델 테스트 Output 데이터 생성 ①**


- 테스트 데이터로 예측 아웃풋 산출 함수 선언

In [ ]:
def sigmoid(x) :
    
    a = np.array(x)

    return a > 0.5

def convert(x) :

    a = []

    for i in x :
      i = 1 if i == [True] else 0
      a.append(i)   

    return a

- 테스트 데이터로 예측 아웃풋 산출 수행

In [ ]:
x_predict_CNN = ()
x_predict_CNN = model.predict(X_test_input)

x_predict_CNN = sigmoid(x_predict_CNN)
x_predict_CNN = x_predict_CNN.tolist()
x_predict_CNN = convert(x_predict_CNN)

x_document = []
x_document = test['document']
x_document = x_document[:10000]
x_document = x_document.tolist()

x_output_CNN = pd.DataFrame({'document' : x_document, 'predict' : x_predict_CNN})

print('아웃풋 데이터: ', x_output_CNN[:100])
print('아웃풋 데이터 Shape: ', np.shape(x_output_CNN))

아웃풋 데이터:                                               document  predict
0                                                 굳 ㅋ        1
1                                GDNTOPCLASSINTHECLUB        1
2              뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아        0
3                    지루하지는 않은데 완전 막장임... 돈주고 보기에는....        0
4   3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??        0
..                                                ...      ...
95                                 전기톱은못들고다니는데 엔진톱이겠죠        0
96                                   완전 재밌엇는데 왜 평점이??        1
97                                     제임스 완이 내 목표임 ㄷ        1
98  1점고 아깝다. 개막장 영화의 원조라고나 할까.아내와 사별한 지 얼마나 지났다고 딴...        0
99  가끔 문득 생각나서 다시보는 영화..색감이 정말 예술이죠 강렬하고 화려하고..츠지야...        1

[100 rows x 2 columns]
아웃풋 데이터 Shape:  (10000, 2)


- CNN 모델 예측 결과 스코어링

In [ ]:
print("CNN model accuracy : ", accuracy_score(labels_test_input, x_predict_CNN))
print("CNN model recall : ",recall_score(labels_test_input, x_predict_CNN))
print("CNN model precision : ",precision_score(labels_test_input, x_predict_CNN))
print("CNN model f1 : ",f1_score(labels_test_input, x_predict_CNN))

CNN model accuracy :  0.8508
CNN model recall :  0.8571428571428571
CNN model precision :  0.8494223614646563
CNN model f1 :  0.8532651455546814


**CNN 예측결과 CSV 파일 저장 ①**

Colab GPU 사용량 문제로 Transformer, CNN, BERT 예측결과 로컬 PC 저장 후 로딩하여 앙상블 결과 산출

In [ ]:
ensemble_CNN = pd.DataFrame({'CNN prediction' : x_predict_CNN})
print(ensemble_CNN)

      CNN prediction
0                  1
1                  1
2                  0
3                  0
4                  0
...              ...
9995               1
9996               0
9997               0
9998               0
9999               0

[10000 rows x 1 columns]


In [ ]:
ensemble_CNN.to_csv('ensemble_CNN.csv')

**BERT 감정분석 데이터 처리 ①,②**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

X_train_tokenized_text = [tokenizer.tokenize(sent) for sent in X_train_bert]
X_test_tokenized_text = [tokenizer.tokenize(sent) for sent in X_test_bert]

print (X_train_tokenized_text[0])
print (X_test_tokenized_text[0])

# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
X_train_bert = [tokenizer.convert_tokens_to_ids(x) for x in X_train_tokenized_text]
X_test_bert = [tokenizer.convert_tokens_to_ids(x) for x in X_test_tokenized_text]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
X_train_bert = pad_sequences(X_train_bert, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
X_test_bert = pad_sequences(X_test_bert, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']
['[CLS]', '굳', '[UNK]', '[SEP]']


**BERT 감정분석 모델 설정 ①,②**

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in X_train_bert:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(X_train_bert,
                                                                                    labels_train, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       X_train_bert,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

attention_masks = []

for seq in X_test_bert:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(X_test_bert)
test_labels = torch.tensor(labels_test)
test_masks = torch.tensor(attention_masks)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 5

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

**BERT 감정분석 모델 학습 실행 ①,②**

- Training, Validation 진행 (Colab Out of memory 시 초기화 후 분할 재실행 필요)

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:01:59.
  Batch 1,000  of  4,219.    Elapsed: 0:03:58.
  Batch 1,500  of  4,219.    Elapsed: 0:05:57.
  Batch 2,000  of  4,219.    Elapsed: 0:07:56.
  Batch 2,500  of  4,219.    Elapsed: 0:09:55.
  Batch 3,000  of  4,219.    Elapsed: 0:11:53.
  Batch 3,500  of  4,219.    Elapsed: 0:13:52.
  Batch 4,000  of  4,219.    Elapsed: 0:15:50.

  Average training loss: 0.38
  Training epcoh took: 0:16:41

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:32

======== Epoch 2 / 5 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:01:59.
  Batch 1,000  of  4,219.    Elapsed: 0:03:57.
  Batch 1,500  of  4,219.    Elapsed: 0:05:55.
  Batch 2,000  of  4,219.    Elapsed: 0:07:53.
  Batch 2,500  of  4,219.    Elapsed: 0:09:52.
  Batch 3,000  of  4,219.    Elapsed: 0:11:50.
  Batch 3,500  of  4,219.    Elapsed: 0:13:49.
  Batch 4,000  of  4,219.    Elapsed: 0:15:48.

  Average training loss: 0.29
  

**BERT 감정분석 모델 테스트 ①,②**

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    BERT_output = []
    BERT_output = BERT_output.append(logits)
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:07.
  Batch   200  of  1,563.    Elapsed: 0:00:13.
  Batch   300  of  1,563.    Elapsed: 0:00:20.
  Batch   400  of  1,563.    Elapsed: 0:00:27.
  Batch   500  of  1,563.    Elapsed: 0:00:34.
  Batch   600  of  1,563.    Elapsed: 0:00:40.
  Batch   700  of  1,563.    Elapsed: 0:00:47.
  Batch   800  of  1,563.    Elapsed: 0:00:54.
  Batch   900  of  1,563.    Elapsed: 0:01:01.
  Batch 1,000  of  1,563.    Elapsed: 0:01:07.
  Batch 1,100  of  1,563.    Elapsed: 0:01:14.
  Batch 1,200  of  1,563.    Elapsed: 0:01:21.
  Batch 1,300  of  1,563.    Elapsed: 0:01:28.
  Batch 1,400  of  1,563.    Elapsed: 0:01:34.
  Batch 1,500  of  1,563.    Elapsed: 0:01:41.

Accuracy: 0.87
Test took: 0:01:45


**BERT 감정분석 모델 신규 텍스트 함수 선언 ①,②**

In [ ]:
# 문장 테스트
def test_sentences(inputs, masks):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    #inputs, masks = convert_input_data(sentences)
    inputs = inputs
    masks = masks

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

**BERT를 이용한 감정분석 모델 테스트 Output 데이터 생성 ①,②** 

- 테스트 데이터 분리 (코랩 GPU 메모리 문제로 분할 실행)

In [ ]:
# Kaggle test 데이터 분리 (Colab GPU 활용 제한으로 분리하여 예측 결과 저장 후 병합)
test_inputs_1 = test_inputs[:2000]
test_masks_1 = test_masks[:2000]
test_inputs_2 = test_inputs[2000:4000]
test_masks_2 = test_masks[2000:4000]
test_inputs_3 = test_inputs[4000:6000]
test_masks_3 = test_masks[4000:6000]
test_inputs_4 = test_inputs[6000:8000]
test_masks_4 = test_masks[6000:8000]
test_inputs_5 = test_inputs[8000:10000]
test_masks_5 = test_masks[8000:10000]

- 테스트 데이터로 예측 수행 1

In [ ]:
logits = test_sentences(test_inputs_1, test_masks_1)

x_predict_BERT_1 = np.argmax(logits, axis = 1)
x_predict_BERT_1 = x_predict_BERT_1.tolist()
print('예측결과:', x_predict_BERT_1[:10])

예측결과: [1, 0, 1, 0, 0, 1, 0, 0, 0, 1]


- BERT output csv 파일 저장 1

In [ ]:
ensemble_BERT_1 = pd.DataFrame({'BERT prediction' : x_predict_BERT_1})
print(ensemble_BERT_1)

      BERT prediction
0                   1
1                   0
2                   1
3                   0
4                   0
...               ...
1995                0
1996                0
1997                0
1998                1
1999                1

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_1.to_csv('ensemble_BERT_1.csv')

- 테스트 데이터로 예측 수행 2

In [ ]:
logits = test_sentences(test_inputs_2, test_masks_2)

x_predict_BERT_2 = np.argmax(logits, axis = 1)
x_predict_BERT_2 = x_predict_BERT_2.tolist()
print('예측결과:', x_predict_BERT_2[:10])

예측결과: [1, 1, 1, 1, 0, 0, 1, 1, 0, 1]


- BERT output csv 파일 저장 2

In [ ]:
ensemble_BERT_2 = pd.DataFrame({'BERT prediction' : x_predict_BERT_2})
print(ensemble_BERT_2)

      BERT prediction
0                   1
1                   1
2                   1
3                   1
4                   0
...               ...
1995                0
1996                0
1997                0
1998                0
1999                1

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_2.to_csv('ensemble_BERT_2.csv')

- 테스트 데이터로 예측 수행 3

In [ ]:
logits = test_sentences(test_inputs_3, test_masks_3)

x_predict_BERT_3 = np.argmax(logits, axis = 1)
x_predict_BERT_3 = x_predict_BERT_3.tolist()
print('예측결과:', x_predict_BERT_3[:10])

예측결과: [1, 1, 1, 0, 0, 0, 0, 0, 1, 1]


- BERT output csv 파일 저장 3

In [ ]:
ensemble_BERT_3 = pd.DataFrame({'BERT prediction' : x_predict_BERT_3})
print(ensemble_BERT_3)

      BERT prediction
0                   1
1                   1
2                   1
3                   0
4                   0
...               ...
1995                0
1996                0
1997                1
1998                0
1999                0

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_3.to_csv('ensemble_BERT_3.csv')

- 테스트 데이터로 예측 수행 4

In [ ]:
logits = test_sentences(test_inputs_4, test_masks_4)

x_predict_BERT_4 = np.argmax(logits, axis = 1)
x_predict_BERT_4 = x_predict_BERT_4.tolist()
print('예측결과:', x_predict_BERT_4[:10])

예측결과: [1, 1, 0, 0, 1, 0, 0, 1, 1, 1]


- BERT output csv 파일 저장 4

In [ ]:
ensemble_BERT_4 = pd.DataFrame({'BERT prediction' : x_predict_BERT_4})
print(ensemble_BERT_4)

      BERT prediction
0                   1
1                   1
2                   0
3                   0
4                   1
...               ...
1995                1
1996                1
1997                0
1998                0
1999                0

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_4.to_csv('ensemble_BERT_4.csv')

- 테스트 데이터로 예측 수행 5

In [ ]:
logits = test_sentences(test_inputs_5, test_masks_5)

x_predict_BERT_5 = np.argmax(logits, axis = 1)
x_predict_BERT_5 = x_predict_BERT_5.tolist()
print('예측결과:', x_predict_BERT_5[:10])

예측결과: [0, 0, 0, 1, 1, 0, 0, 1, 0, 1]


- BERT output csv 파일 저장 5

In [ ]:
ensemble_BERT_5 = pd.DataFrame({'BERT prediction' : x_predict_BERT_5})
print(ensemble_BERT_5)

      BERT prediction
0                   0
1                   0
2                   0
3                   1
4                   1
...               ...
1995                1
1996                0
1997                0
1998                0
1999                1

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_5.to_csv('ensemble_BERT_5.csv')

**BERT 예측결과 CSV 파일 저장**

In [ ]:
ensemble_BERT = np.concatenate((x_predict_BERT_1, x_predict_BERT_2, x_predict_BERT_3, x_predict_BERT_4, x_predict_BERT_5), axis = 0)

In [ ]:
ensemble_BERT_out = pd.DataFrame({'BERT prediction' : ensemble_BERT})
print(ensemble_BERT_out)

In [ ]:
ensemble_BERT_out.to_csv('ensemble_BERT.csv')

※ 본 셀은 GPU 사용량 문제로 BERT 분할 예측결과 CSV 파일 업로드 시에만 사용

- 업로드 ensemble_BERT_1, ensemble_BERT_2, ensemble_BERT_3, ensemble_BERT_4, ensemble_BERT_5

In [ ]:
from google.colab import files
files.upload()

Saving ensemble_BERT_1.csv to ensemble_BERT_1.csv
Saving ensemble_BERT_2.csv to ensemble_BERT_2.csv
Saving ensemble_BERT_3.csv to ensemble_BERT_3.csv
Saving ensemble_BERT_4.csv to ensemble_BERT_4.csv
Saving ensemble_BERT_5.csv to ensemble_BERT_5.csv


{'ensemble_BERT_1.csv': b',BERT prediction\n0,1\n1,0\n2,1\n3,0\n4,0\n5,1\n6,0\n7,0\n8,0\n9,1\n10,1\n11,0\n12,1\n13,1\n14,0\n15,1\n16,1\n17,0\n18,0\n19,0\n20,1\n21,0\n22,1\n23,1\n24,0\n25,0\n26,1\n27,1\n28,0\n29,1\n30,1\n31,1\n32,0\n33,1\n34,0\n35,0\n36,0\n37,0\n38,0\n39,0\n40,0\n41,0\n42,0\n43,1\n44,1\n45,1\n46,1\n47,0\n48,1\n49,1\n50,0\n51,0\n52,0\n53,1\n54,1\n55,1\n56,1\n57,0\n58,1\n59,1\n60,0\n61,0\n62,0\n63,0\n64,1\n65,0\n66,1\n67,1\n68,1\n69,1\n70,0\n71,0\n72,0\n73,0\n74,0\n75,1\n76,1\n77,1\n78,1\n79,0\n80,0\n81,1\n82,0\n83,1\n84,0\n85,1\n86,1\n87,0\n88,1\n89,1\n90,0\n91,1\n92,0\n93,1\n94,0\n95,0\n96,1\n97,1\n98,0\n99,1\n100,0\n101,0\n102,1\n103,1\n104,1\n105,0\n106,1\n107,1\n108,0\n109,1\n110,1\n111,0\n112,1\n113,1\n114,1\n115,0\n116,1\n117,1\n118,1\n119,1\n120,0\n121,0\n122,1\n123,1\n124,0\n125,1\n126,0\n127,1\n128,0\n129,1\n130,0\n131,0\n132,0\n133,1\n134,1\n135,1\n136,1\n137,1\n138,1\n139,1\n140,0\n141,0\n142,1\n143,1\n144,0\n145,1\n146,1\n147,0\n148,0\n149,1\n150,1\n151,0\n15

In [ ]:
ensemble_BERT_1 = pd.read_csv("./ensemble_BERT_1.csv")
ensemble_BERT_2 = pd.read_csv("./ensemble_BERT_2.csv")
ensemble_BERT_3 = pd.read_csv("./ensemble_BERT_3.csv")
ensemble_BERT_4 = pd.read_csv("./ensemble_BERT_4.csv")
ensemble_BERT_5 = pd.read_csv("./ensemble_BERT_5.csv")

In [ ]:
ensemble_BERT_1_out = ensemble_BERT_1['BERT prediction']
ensemble_BERT_2_out = ensemble_BERT_2['BERT prediction']
ensemble_BERT_3_out = ensemble_BERT_3['BERT prediction']
ensemble_BERT_4_out = ensemble_BERT_4['BERT prediction']
ensemble_BERT_5_out = ensemble_BERT_5['BERT prediction']

In [ ]:
ensemble_BERT_sum = ensemble_BERT_1_out
ensemble_BERT_sum = ensemble_BERT_sum.append(ensemble_BERT_2_out)
ensemble_BERT_sum = ensemble_BERT_sum.append(ensemble_BERT_3_out)
ensemble_BERT_sum = ensemble_BERT_sum.append(ensemble_BERT_4_out)
ensemble_BERT_sum = ensemble_BERT_sum.append(ensemble_BERT_5_out)

In [ ]:
ensemble_BERT_sum_out = ensemble_BERT_sum.tolist()

In [ ]:
ensemble_BERT = pd.DataFrame({'BERT prediction' : ensemble_BERT_sum_out})
print(ensemble_BERT)

      BERT prediction
0                   1
1                   0
2                   1
3                   0
4                   0
...               ...
9995                1
9996                0
9997                0
9998                0
9999                1

[10000 rows x 1 columns]


In [ ]:
ensemble_BERT.to_csv('ensemble_BERT.csv')

- BERT 모델 예측 결과 스코어링

In [ ]:
labels_test_input = labels_test[:10000]

In [ ]:
print("BERT model accuracy : ", accuracy_score(labels_test_input, ensemble_BERT_sum_out))
print("BERT model recall : ",recall_score(labels_test_input, ensemble_BERT_sum_out))
print("BERT model precision : ",precision_score(labels_test_input, ensemble_BERT_sum_out))
print("BERT model f1 : ",f1_score(labels_test_input, ensemble_BERT_sum_out))

BERT model accuracy :  0.8711
BERT model recall :  0.8782849239280774
BERT model precision :  0.8685033216100039
BERT model f1 :  0.8733667354357009


**앙상블 예측 결과 산출 ③**

- Colab GPU 사용량 문제로 Transformer, CNN, BERT 예측결과 로컬 PC 저장 후 로딩하여 앙상블 결과 산출

- Transformer(ensemble_transformer.csv), CNN(ensemble_CNN.csv), BERT(ensemble_BERT.csv) 파일 로컬 PC에서 업로드

In [ ]:
from google.colab import files
files.upload()

Saving ensemble_BERT.csv to ensemble_BERT (1).csv
Saving ensemble_CNN.csv to ensemble_CNN.csv
Saving ensemble_transformer.csv to ensemble_transformer.csv


{'ensemble_BERT.csv': b',BERT prediction\n0,1\n1,0\n2,1\n3,0\n4,0\n5,1\n6,0\n7,0\n8,0\n9,1\n10,1\n11,0\n12,1\n13,1\n14,0\n15,1\n16,1\n17,0\n18,0\n19,0\n20,1\n21,0\n22,1\n23,1\n24,0\n25,0\n26,1\n27,1\n28,0\n29,1\n30,1\n31,1\n32,0\n33,1\n34,0\n35,0\n36,0\n37,0\n38,0\n39,0\n40,0\n41,0\n42,0\n43,1\n44,1\n45,1\n46,1\n47,0\n48,1\n49,1\n50,0\n51,0\n52,0\n53,1\n54,1\n55,1\n56,1\n57,0\n58,1\n59,1\n60,0\n61,0\n62,0\n63,0\n64,1\n65,0\n66,1\n67,1\n68,1\n69,1\n70,0\n71,0\n72,0\n73,0\n74,0\n75,1\n76,1\n77,1\n78,1\n79,0\n80,0\n81,1\n82,0\n83,1\n84,0\n85,1\n86,1\n87,0\n88,1\n89,1\n90,0\n91,1\n92,0\n93,1\n94,0\n95,0\n96,1\n97,1\n98,0\n99,1\n100,0\n101,0\n102,1\n103,1\n104,1\n105,0\n106,1\n107,1\n108,0\n109,1\n110,1\n111,0\n112,1\n113,1\n114,1\n115,0\n116,1\n117,1\n118,1\n119,1\n120,0\n121,0\n122,1\n123,1\n124,0\n125,1\n126,0\n127,1\n128,0\n129,1\n130,0\n131,0\n132,0\n133,1\n134,1\n135,1\n136,1\n137,1\n138,1\n139,1\n140,0\n141,0\n142,1\n143,1\n144,0\n145,1\n146,1\n147,0\n148,0\n149,1\n150,1\n151,0\n152,

In [ ]:
ensemble_transformer = pd.read_csv("./ensemble_transformer.csv")
ensemble_CNN= pd.read_csv("./ensemble_CNN.csv")
ensemble_BERT = pd.read_csv("./ensemble_BERT.csv")


print(ensemble_transformer)
print(ensemble_CNN)
print(ensemble_BERT)

      Unnamed: 0  transformer prediction
0              0                       1
1              1                       0
2              2                       0
3              3                       0
4              4                       0
...          ...                     ...
9995        9995                       1
9996        9996                       0
9997        9997                       0
9998        9998                       0
9999        9999                       0

[10000 rows x 2 columns]
      Unnamed: 0  CNN prediction
0              0               1
1              1               1
2              2               0
3              3               0
4              4               0
...          ...             ...
9995        9995               1
9996        9996               0
9997        9997               0
9998        9998               0
9999        9999               0

[10000 rows x 2 columns]
      Unnamed: 0  BERT prediction
0              0          

In [ ]:
ensemble_transformer_out = ensemble_transformer['transformer prediction']
ensemble_CNN_out= ensemble_CNN['CNN prediction']
ensemble_BERT_out = ensemble_BERT['BERT prediction']

In [ ]:
x_output_total = pd.DataFrame({'Transformer' : ensemble_transformer_out, 'CNN' : ensemble_CNN_out, 'BERT' : ensemble_BERT_out})

print(x_output_total)

      Transformer  CNN  BERT
0               1    1     1
1               0    1     0
2               0    0     1
3               0    0     0
4               0    0     0
...           ...  ...   ...
9995            1    1     1
9996            0    0     0
9997            0    0     0
9998            0    0     0
9999            0    0     1

[10000 rows x 3 columns]


- 앙상블 예측결과 Hard Voting

In [ ]:
s = np.sum(x_output_total, axis = 1)
s = s.tolist()
print(s)

[3, 1, 1, 0, 0, 3, 0, 0, 0, 3, 3, 0, 3, 3, 0, 1, 3, 0, 2, 0, 3, 0, 3, 1, 0, 2, 2, 3, 1, 3, 3, 2, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 3, 0, 3, 2, 0, 2, 0, 2, 3, 3, 2, 2, 3, 3, 0, 0, 0, 0, 3, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 3, 0, 3, 0, 3, 3, 0, 3, 1, 0, 3, 0, 3, 0, 1, 3, 2, 0, 3, 0, 1, 3, 3, 3, 0, 3, 1, 0, 3, 1, 2, 3, 3, 3, 0, 1, 3, 3, 3, 2, 0, 3, 3, 0, 3, 0, 3, 0, 3, 0, 2, 0, 3, 3, 1, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 2, 1, 2, 0, 3, 3, 3, 0, 0, 0, 1, 3, 3, 0, 3, 1, 0, 3, 2, 3, 0, 0, 2, 3, 0, 0, 0, 3, 2, 0, 3, 3, 3, 0, 2, 3, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 1, 1, 3, 1, 3, 0, 1, 3, 3, 1, 1, 3, 1, 0, 0, 0, 3, 3, 1, 3, 2, 0, 0, 3, 2, 3, 0, 1, 3, 0, 0, 0, 1, 3, 3, 0, 3, 3, 3, 3, 0, 0, 0, 1, 0, 0, 2, 3, 3, 1, 3, 0, 3, 3, 3, 2, 1, 3, 0, 3, 3, 3, 3, 1, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 2, 1, 3, 1, 3, 0, 2, 0, 3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 1, 3, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 3, 1, 2, 0, 1, 3, 0, 3, 3, 2, 

- 앙상블 예측결과 산출 (각 모델의 예측결과 합이 >= 2 면 1, 아니면 0)

In [ ]:
predict_ensemble = []

for i in s :
      i = 1 if i >= 2 else 0
      predict_ensemble.append(i)   

print(predict_ensemble)

[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 

- 앙상블 예측결과와 테스트 레이블 데이터 결합

In [ ]:
ensemble_submission_w_label = pd.DataFrame({'Predicted' : predict_ensemble, 'label' : labels_test_input})
print(ensemble_submission_w_label)

      Predicted  label
0             1      1
1             0      0
2             0      0
3             0      0
4             0      0
...         ...    ...
9995          1      1
9996          0      0
9997          0      0
9998          0      0
9999          0      0

[10000 rows x 2 columns]


- 앙상블 예측결과와 테스트 레이블 데이터 결합 인스턴스 CSV 파일 저장(필요 시)

In [ ]:
ensemble_submission_w_label.to_csv('ensemble_predict.csv')

- 앙상블 모델 예측 결과 스코어링

In [ ]:
print("앙상블 model accuracy : ", accuracy_score(labels_test_input, predict_ensemble))
print("앙상블 model recall : ",recall_score(labels_test_input,predict_ensemble))
print("앙상블 model precision : ",precision_score(labels_test_input, predict_ensemble))
print("앙상블 model f1 : ",f1_score(labels_test_input, predict_ensemble))

앙상블 model accuracy :  0.8745
앙상블 model recall :  0.8691958111045248
앙상블 model precision :  0.8812099358974359
앙상블 model f1 :  0.87516164329056
